### Importing libraries

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from aix360.algorithms.protodash import ProtodashExplainer
from src.utils.data_utils import read_from_file

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

artifact_path = "artifacts"
results_path = "results"


In [ ]:
# set plot parameters
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=12)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=12)  # fontsize of the tick labels
plt.rc("ytick", labelsize=12)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title


### Load up artifacts

In [ ]:
artifacts = read_from_file(f"{artifact_path}/adult_dataset_artifacts.p")
model_ids = read_from_file(f"{artifact_path}/adult_model_ids.p")


# unpack from artifacts
clf = artifacts["clf"]

conformal_dict = artifacts["conformal_dict"]
inliers_dict = artifacts["inliers_dict"]
outliers_dict = artifacts["outliers_dict"]
benchmark_mean = artifacts["benchmark_mean"]
benchmark_std = artifacts["benchmark_std"]
small_ci_ids = artifacts["small_ci_ids"]
large_ci_ids = artifacts["large_ci_ids"]
df_out = artifacts["df_out"]
X_train = artifacts["X_train"]
y_train = artifacts["y_train"]
X_test = artifacts["X_test"]
y_test = artifacts["y_test"]
X = artifacts["X"]


In [ ]:
benchmark_mean

# MPI RESULTS

In [ ]:
MPI_df = pd.DataFrame([benchmark_mean, benchmark_std], index=["Mean", "Std"])

MPI_df.to_csv(f"{results_path}/adult/MPI_df.csv")

MPI_df


## Group feature comparison

### Run Protodash as a comparison & assess correlation

In [ ]:
explainer = ProtodashExplainer()

m = 100
(W, S, setValues) = explainer.explain(
    X_test, X_train, m=m, kernelType="Gaussian", sigma=2
)


In [ ]:
from scipy import stats

cert_ids = small_ci_ids[0:100]
uncert_ids = large_ci_ids[-100:]

vals = []
for i in range(X_train.shape[1]):
    vals.append(
        stats.spearmanr(np.sort(X_train[S, i]), np.sort(X_test[cert_ids, i]))[0]
    )

print(f"Correlation between prototypes vs certain: {np.nanmean(vals)}")


In [ ]:
vals = []
for i in range(X_train.shape[1]):
    vals.append(
        np.corrcoef(np.sort(X_train[S, i]), np.sort(X_test[uncert_ids, i]))[0, 1]
    )

print(f"Correlation between prototypes vs uncertain: {np.nanmean(vals)}")


### Compare the features of the various groups - data insights

In [ ]:
cert_ids = small_ci_ids[0:100]
uncert_ids = large_ci_ids[-100:]

uncert_mean = np.mean(X_test[uncert_ids, :], axis=0)
cert_mean = np.mean(X_test[cert_ids, :], axis=0)
train_mean = np.mean(X_train, axis=0)
test_mean = np.mean(X_test, axis=0)
cert_mean = np.mean(X_test[cert_ids, :], axis=0)

proto_mean = np.mean(X_train[S, :], axis=0)


# comparison_df = pd.DataFrame({'proto': proto_mean, 'certain': cert_mean,'uncertain':uncert_mean, "train": train_mean, "test": test_mean}, index=X.columns)
comparison_df = pd.DataFrame(
    {
        "certain": cert_mean,
        "uncertain": uncert_mean,
        "protodash": proto_mean,
        "train": train_mean,
        "test": test_mean,
    },
    index=X.columns,
)

comparison_df.to_csv(f"{results_path}/adult/comparison_df.csv")

comparison_df


# Data-insights figure

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

train_sc = scaler.transform(X_train)
test_sc = scaler.transform(X_test)

pca = PCA(n_components=2)
pcs_train = pca.fit_transform(train_sc)
pcs_test = pca.transform(test_sc)


n_ids = 100
cert_ids = small_ci_ids[0:n_ids]  # certain[0:100]
uncert_ids = large_ci_ids[-n_ids:]

n_ids = 100
# pca.transform(np.mean(X_test[uncert_ids,:], axis=0).reshape(1, -1))
uncert_mean = np.mean(pcs_test[large_ci_ids[-n_ids:], :], axis=0)
# pca.transform(np.mean(X_test[cert_ids,:], axis=0).reshape(1, -1))
cert_mean = np.mean(pcs_test[small_ci_ids[0:n_ids], :], axis=0)
# proto_mean = np.mean(pcs_test[S,:],axis=0) #pca.transform(np.mean(X_test[cert_ids,:], axis=0).reshape(1, -1))


plt.scatter(pcs_train[:, 0], pcs_train[:, 1], color="0.75", label="Train")
plt.scatter(
    pcs_test[cert_ids, 0], pcs_test[cert_ids, 1], color="g", label="Test - Certain"
)
plt.scatter(
    pcs_test[uncert_ids, 0],
    pcs_test[uncert_ids, 1],
    color="r",
    label="Test - Uncertain",
)
# plt.scatter(pcs_test[S,0],pcs_test[S,1], color='b', label = 'Test - s')
plt.scatter(cert_mean[0], cert_mean[1], color="k", s=500, marker="*")
plt.scatter(uncert_mean[0], uncert_mean[1], color="k", s=300, marker="X")

# plt.scatter(proto_mean[0],proto_mean[1], color='b', s=300, marker='X')
plt.xlabel("X1")
plt.ylabel("X2")
plt.title("Samples per group: 100")
plt.legend()

plt.savefig(f"{results_path}/adult/adult_insights_diagram.png")


## Lambda sweep plot (flag examples)

In [ ]:
def get_sort_dict(outliers_dict):
    counts_dict = {}
    for key in outliers_dict.keys():
        out_array = outliers_dict[key]

        for val in out_array:
            if val not in counts_dict.keys():
                counts_dict[val] = 1
            else:
                counts_dict[val] += 1

    sort_dict = {k: v for k, v in sorted(counts_dict.items(), key=lambda item: item[1])}
    return sort_dict


def inconsistent_lambda_sweep(outliers_dict, X_test, y_test, clf):

    incons_scores = []
    total_samples = []

    sort_dict = get_sort_dict(outliers_dict)

    for ns in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        flagged_incons_ids = []
        for key in list(sort_dict.keys()):
            if sort_dict[key] > ns:
                flagged_incons_ids.append(key)

        flagged_incons_ids = np.array(flagged_incons_ids)

        flagged_incons_ids

        myids = flagged_incons_ids
        y_true = y_test
        y_pred = clf.predict(X_test[myids, :])
        total_samples.append(len(flagged_incons_ids))

        incons_scores.append(accuracy_score(y_true[myids], y_pred))

    return incons_scores, total_samples, sort_dict


incons_scores, total_samples, sort_dict = inconsistent_lambda_sweep(
    outliers_dict, X_test, y_test, clf
)


plt.plot(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) / (X_test.shape[1]), incons_scores)
plt.ylabel("Accuracy Score")
plt.xlabel("Proportion of features - λ")


axes1 = plt.gca()
axes2 = axes1.twiny()

axes2.set_xticklabels(np.array(total_samples)[3:])

axes2.set_xlabel("n samples")
axes1.set_xlabel("Proportion of features - λ")
plt.savefig(f"{results_path}/adult/adult_lambda_sweep.png")


### OOD COMPARISON EXPERIMENT - OVERLAP

In [ ]:
def ood_test(X_train, X_test, sort_dict, large_ci_ids, n_sample_threshold=8):

    flagged_incons_ids = []
    for key in list(sort_dict.keys()):
        if sort_dict[key] > n_sample_threshold:
            flagged_incons_ids.append(key)

    flagged_incons_ids = np.array(flagged_incons_ids)

    results = {}
    for samps in [50, 100, 150, 200, 250]:
        print("SAMPLING: ", samps)
        n_ids = samps
        uncert_ids = large_ci_ids[-n_ids:]
        uncert_ids.shape

        from pyod.models.iforest import IForest
        from pyod.models.copod import COPOD
        from pyod.models.lof import LOF

        from pyod.models.suod import SUOD

        # # initialized a group of outlier detectors for acceleration
        detector_list = [
            LOF(n_neighbors=15),
            LOF(n_neighbors=20),
            LOF(n_neighbors=25),
            LOF(n_neighbors=35),
            COPOD(),
            IForest(n_estimators=100, random_state=42),
            IForest(n_estimators=200, random_state=42),
        ]

        # # decide the number of parallel process, and the combination method
        # # then clf can be used as any outlier detection model

        clfs = [
            COPOD(),
            IForest(n_estimators=100, random_state=42),
            SUOD(
                base_estimators=detector_list,
                n_jobs=2,
                combination="average",
                verbose=False,
            ),
        ]

        means = []
        stds = []
        uncert_ood = []
        incons_ood = []
        clf_id = 0
        for clf_current in clfs:
            clf_id += 1
            print(clf_id)
            clf = clf_current
            clf.fit(X_train)
            probas = clf.predict_proba(X_test)[:, 1]
            means.append(np.mean(probas))
            stds.append(np.std(probas))

            ood = np.argsort(probas)[-len(uncert_ids) :]
            i = 0
            for val in ood:
                if val in uncert_ids:
                    i += 1

            uncert_ood.append(i / len(ood))

            ood = np.argsort(probas)[-len(flagged_incons_ids) :]
            i = 0
            for val in ood:
                if val in flagged_incons_ids:
                    i += 1

            incons_ood.append(i / len(ood))

        results[samps] = {
            "means": means,
            "stds": stds,
            "uncert_ood": uncert_ood,
            "incons_ood": incons_ood,
        }

    from alibi_detect.od.mahalanobis import Mahalanobis

    od = Mahalanobis(cat_vars={3: 1, 4: 3, 5: 3, 6: 1, 7: 1, 8: 1, 10: 1, 11: 2})
    od.fit(X_train)

    scores = od.score(X_test)

    mahalanobis_incons = []
    mahalanobis_uncert = []
    for samps in [50, 100, 150, 200, 250]:
        print("SAMPLING: ", samps)
        n_ids = samps
        uncert_ids = large_ci_ids[-n_ids:]
        ood = np.argsort(scores)[-len(flagged_incons_ids) :]
        i = 0
        for val in ood:
            if val in flagged_incons_ids:
                i += 1
        mahalanobis_incons.append(i / len(ood))

        ood = np.argsort(scores)[-len(uncert_ids) :]
        i = 0
        for val in ood:
            if val in uncert_ids:
                i += 1
        mahalanobis_uncert.append(i / len(ood))

    return results, mahalanobis_incons, mahalanobis_uncert


results, mahalanobis_incons, mahalanobis_uncert = ood_test(
    X_train, X_test, sort_dict, large_ci_ids, n_sample_threshold=8
)


In [ ]:
sample_arr = []
method1_ood = []
method2_ood = []
method3_ood = []

method1_mean = []
method2_mean = []
method3_mean = []

method1_std = []
method2_std = []
method3_std = []

method1_incons = []
method2_incons = []
method3_incons = []


for samps in [50, 100, 150, 200, 250]:

    sample_arr.append(samps)

    method1_ood.append(results[samps]["uncert_ood"][0])
    method2_ood.append(results[samps]["uncert_ood"][1])
    method3_ood.append(results[samps]["uncert_ood"][2])

    method1_incons.append(results[samps]["incons_ood"][0])
    method2_incons.append(results[samps]["incons_ood"][1])
    method3_incons.append(results[samps]["incons_ood"][2])

    method1_mean.append(results[samps]["means"][0])
    method2_mean.append(results[samps]["means"][1])
    method3_mean.append(results[samps]["means"][2])

    method1_std.append(results[samps]["stds"][0])
    method2_std.append(results[samps]["stds"][1])
    method3_std.append(results[samps]["stds"][2])


### OOD COMPARISON PLOTS

In [ ]:
plt.plot(sample_arr[:-1], method1_ood[:-1], marker="o", label="COPOD")
plt.plot(sample_arr[:-1], method2_ood[:-1], marker="o", label="IForest")
plt.plot(sample_arr[:-1], method3_ood[:-1], marker="o", label="SUOD")
plt.plot(sample_arr[:-1], mahalanobis_uncert[:-1], marker="o", label="Mahalanobis")
plt.xlabel("Top n uncertain samples")
plt.ylabel("OOD-Uncertain Sample Match Rate")
plt.legend()
plt.savefig(f"{results_path}/adult/adult_ood_uncertain_match.png")


In [ ]:
plt.plot(sample_arr, method1_mean, marker="o", label="COPOD")
plt.plot(sample_arr, method2_mean, marker="o", label="IForest")
plt.plot(sample_arr, method3_mean, marker="o", label="SUOD")
plt.xlabel("Top n uncertain samples")
plt.ylabel("Mean Probability of potential OODs")
plt.legend()
plt.savefig(f"{results_path}/adult/adult_ood_proba.png")


In [ ]:
plt.plot(sample_arr[:], method1_incons[:], marker="o", label="COPOD")
plt.plot(sample_arr[:], method2_incons[:], marker="o", label="IForest")
plt.plot(sample_arr[:], method3_incons[:], marker="o", label="SUOD")
plt.plot(sample_arr[:], mahalanobis_incons[:], marker="o", label="Mahalanobis")
plt.xlabel("Top n inconsistent samples")
plt.ylabel("OOD-Inconsistent Sample Match Rate")
plt.legend()
plt.savefig(f"{results_path}/adult/adult_incons_uncertain_match.png")


# Diverse downstream model experiment

In [ ]:
import logging

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.svm import SVC

from aix360.algorithms.protodash import ProtodashExplainer
from sklearn_extra.robust import RobustWeightedClassifier

logger = logging.getLogger()
logger.setLevel(logging.INFO)

model_compare = {}

model_list = ["rf", "mlp", "gbt", "robust"]


for downstream_model in model_list:

    logging.info(f"Testing downstream - {downstream_model}")
    benchmark_mean = {}
    benchmark_std = {}
    seed = 0

    #################################
    # Train diverse models
    #################################
    if downstream_model == "rf":
        clf = RandomForestClassifier()
    if downstream_model == "mlp":
        clf = MLPClassifier(random_state=seed)
    if downstream_model == "gbt":
        clf = GradientBoostingClassifier()
    if downstream_model == "robust":
        # Scale the dataset with sklearn RobustScaler (important for this algorithm)

        clf = RobustWeightedClassifier(
            weighting="huber",
            loss="hinge",
            c=1.35,
            eta0=1e-3,
            max_iter=300,
        )

    small_ci_ids = artifacts["small_ci_ids"]
    large_ci_ids = artifacts["large_ci_ids"]

    if downstream_model != "robust":
        clf.fit(X_train, y_train)
    else:
        from copy import deepcopy

        Xr = deepcopy(X_train)
        sc = RobustScaler()
        Xr = sc.fit_transform(Xr)
        clf.fit(Xr, y_train)
        Xt = deepcopy(X_test)
        Xt = sc.transform(Xt)

    #################################
    # Predict for the different CI levels
    #################################
    mean_cert = []
    for n_ids in range(100, 500, 100):
        myids = small_ci_ids[0:n_ids]

        y_true = y_test
        if downstream_model != "robust":
            y_pred = clf.predict(X_test[myids, :])
        else:
            y_pred = clf.predict(Xt[myids, :])

        acc_sc = accuracy_score(y_true[myids], y_pred)
        mean_cert.append(acc_sc)

    mean_cert = np.array(mean_cert)

    mean_uncert = []

    for n_ids in range(100, 500, 100):
        myids = large_ci_ids[-n_ids:]

        y_true = y_test
        if downstream_model != "robust":
            y_pred = clf.predict(X_test[myids, :])
        else:
            y_pred = clf.predict(Xt[myids, :])

        acc_sc = accuracy_score(y_true[myids], y_pred)
        mean_uncert.append(acc_sc)

    mean_uncert = np.array(mean_uncert)

    diff_mean = np.mean(mean_cert - mean_uncert)
    diff_std = np.std(mean_cert - mean_uncert)

    if downstream_model != "robust":
        benchmark_mean["Data-SUITE"] = diff_mean
        benchmark_std["Data-SUITE"] = diff_std
    else:
        benchmark_mean["Data-SUITE"] = np.abs(diff_mean)
        benchmark_std["Data-SUITE"] = np.abs(diff_std)

    #################################
    # Assess all other baselines
    #################################
    comparison_models = ["qr", "bnn", "conformal", "mcd", "ensemble", "gp"]

    for model in comparison_models:
        ordered_scores = model_ids[model]
        model_certainty = []
        model_uncertainty = []

        samples = np.arange(100, 500, 100)
        for sample in samples:
            certain = ordered_scores[0:sample]
            uncertain = ordered_scores[-sample:]

            if downstream_model != "robust":
                y_pred = clf.predict(X_test[certain, :])
            else:
                y_pred = clf.predict(Xt[certain, :])
            model_certainty.append(accuracy_score(y_test[certain], y_pred))

            if downstream_model != "robust":
                y_pred = clf.predict(X_test[uncertain, :])
            else:
                y_pred = clf.predict(Xt[uncertain, :])

            model_uncertainty.append(accuracy_score(y_test[uncertain], y_pred))

        diff_mean = np.mean(np.array(model_certainty) - np.array(model_uncertainty))
        diff_std = np.std(np.array(model_certainty) - np.array(model_uncertainty))

        if downstream_model != "robust":
            benchmark_mean[model] = diff_mean
            benchmark_std[model] = diff_std
        else:
            benchmark_mean[model] = np.abs(diff_mean)
            benchmark_std[model] = np.abs(diff_std)

    rank = sorted(benchmark_mean, key=benchmark_mean.get)

    model_compare[downstream_model] = rank


In [ ]:
def process_model_comparison(model_compare):
    baselines = ["ensemble", "gp", "bnn", "conformal", "qr", "mcd", "Data-SUITE"]

    results = {}

    for baseline in baselines:
        scores = []
        for model in model_list:
            index = model_compare[model][::-1].index(baseline) + 1
            scores.append(index)
        results[baseline] = scores
    return results


results = process_model_comparison(model_compare)


In [ ]:
# PLOT RESULTS

SMALL_SIZE = 10
MEDIUM_SIZE = 50
BIGGER_SIZE = 50

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=12)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=12)  # fontsize of the tick labels
plt.rc("ytick", labelsize=12)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title


plt.figure(figsize=(6, 4))
plt.grid()

model_list = [mod.upper() for mod in model_list]

for key in results.keys():
    if key == "Data-SUITE":
        w = 4
    else:
        w = 1.5
    plt.plot(model_list, results[key], marker="o", label=key.upper(), linewidth=w)

plt.gca().invert_yaxis()
plt.ylabel("MPI rank (HIGHER BETTER)", fontweight="bold", fontsize=14)
plt.xlabel("Downstream model type", fontweight="bold", fontsize=14)
plt.xticks(fontsize=15, fontweight="bold")
plt.yticks(fontsize=15, fontweight="bold")
leg = plt.legend()
_, _, _, _, _, _, ds = leg.get_texts()
ds.set_size(16)
plt.savefig(f"{results_path}/adult/adult_diverse_model_mpi.png")


# MPI with different downstream models

In [ ]:
downstream_model = "rf"

if downstream_model == "rf":
    clf = RandomForestClassifier()
if downstream_model == "mlp":
    clf = MLPClassifier()
if downstream_model == "gbt":
    clf = GradientBoostingClassifier()
if downstream_model == "robust":
    # Scale the dataset with sklearn RobustScaler (important for this algorithm)

    clf = RobustWeightedClassifier(
        weighting="mom",
        loss="hinge",
        c=1.35,
        eta0=1e-3,
        max_iter=300,
    )
if downstream_model == "robust":
    # Scale the dataset with sklearn RobustScaler (important for this algorithm)

    clf = RobustWeightedClassifier(
        weighting="huber",
        loss="hinge",
        c=1.35,
        eta0=1e-3,
        max_iter=300,
    )
clf.fit(X_train, y_train)


mean_cert = []

for n_ids in range(100, 500, 100):
    myids = small_ci_ids[0:n_ids]

    y_true = y_test
    y_pred = clf.predict(X_test[myids, :])

    acc_sc = accuracy_score(y_true[myids], y_pred)
    mean_cert.append(acc_sc)

mean_cert = np.array(mean_cert)

mean_uncert = []

for n_ids in range(100, 500, 100):
    myids = large_ci_ids[-n_ids:]

    y_true = y_test
    y_pred = clf.predict(X_test[myids, :])

    acc_sc = accuracy_score(y_true[myids], y_pred)
    mean_uncert.append(acc_sc)

mean_uncert = np.array(mean_uncert)


mean_mpi = np.mean(mean_cert - mean_uncert)
std_mpi = np.std(mean_cert - mean_uncert)

print(f"MPI = {mean_mpi} +- {std_mpi}")
